# **BICEP CURL DETECTOR**


**Shankara Abbineni**

This is a personal project aimed at combining my interest in computer vision with my passion for exercise. A common problem faced by many individuals in the gym is forgetting to keep track of how many repetitions of an exercise have been completed. Often, it is more helpful to not focus on the number of repetitions but instead on the quality of the movement. As such, in order to solve this problem (to some extent), this project leverages the capabilities of OpenCV and MediaPipe to count the number of repetitions for bicep curls. In the future, I plan on expanding this basic structure to more exercises, including push-ups and pull-ups.

## Import Dependencies

As mentioned in the introduction, the packages we are using in this project are OpenCV for camera data, MediaPipe for pose estimation, and NumPy for calculating angles.

In [2]:
import cv2
import mediapipe as mp
import numpy as np

## Detection System

In this section, we will use OpenCV to pull a video feed and MediaPipe to extract landmarks. Then, using MediaPipe's drawing utilities, we can draw on the joints and connections.

In [3]:
mp_drawing = mp.solutions.drawing_utils  # Getting all drawing utilites
mp_pose = mp.solutions.pose  # Imports the pose estimation model

In [ ]:
# Testing Camera Feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Webcam Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):  # Pressing 'q' stops the feed
        break

cap.release()
cv2.destroyAllWindows()  # Close the OpenCV window
cv2.waitKey(1)

In [3]:
cap = cv2.VideoCapture(0)

# Setting up a MediaPipe instance
with mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change cv BGR image to mp RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Making detections
        results = pose.process(image)

        # Turning RGB back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Rendering detections on frame
        # Passing through image, landmarks from results, which landmark is connected to which, joint drawing spec, connection drawing spec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(252, 80, 3), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(3, 177, 252), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Joint Determination

Now, we can extract information for the joints that matter in a bicep curl: the wrist, elbow, and shoulder. Let's focus on doing this for only the left arm and include the right arm later on.

In [4]:
cap = cv2.VideoCapture(0)

# Setting up a MediaPipe instance
with mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change cv BGR image to mp RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Making detections
        results = pose.process(image)

        # Turning RGB back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Rendering detections on frame
        # Passing through image, landmarks from results, which landmark is connected to which, joint drawing spec, connection drawing spec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(252, 80, 3), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(3, 177, 252), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [19]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.7348299622535706
y: 0.40620389580726624
z: -0.46189093589782715
visibility: 0.9992040991783142

In [20]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.9523245692253113
y: 0.819460391998291
z: -0.3104078471660614
visibility: 0.9756779074668884

In [21]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7348299622535706
y: 0.40620389580726624
z: -0.46189093589782715
visibility: 0.9992040991783142

## Angle Calculation

Since we are able to extract the coordinates of the specified joints, we can use the $\tan$ function to find the angle of the arm at the elbow.

In [4]:
def calcAngle(a, b, c):
    a = np.array(a)  # First Joint
    b = np.array(b)  # Middle Joint (main vertex)
    c = np.array(c)  # Third Joint

    radians = np.arctan2(a[1]-b[1], a[0]-b[0]) - np.arctan2(c[1]-b[1], c[0]-b[0])
    degrees = np.abs(radians * 180.0 / np.pi)  # Coverting radians to degrees

    if degrees > 180.0:
        degrees = 360 - degrees

    return degrees

In [6]:
left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

In [12]:
calcAngle(left_wrist, left_elbow, left_shoulder)

159.73819208026083

In [19]:
# Details about webcam resolution
# A MacBook Air M1 has a webcam resolution of 1280x720

webcam_width = 1280
webcam_height = 720

In [20]:
cap = cv2.VideoCapture(0)

# Setting up a MediaPipe instance
with mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change cv BGR image to mp RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Making detections
        results = pose.process(image)

        # Turning RGB back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Grab coordinates of left wrist, left elbow, and left shoulder
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Calculate angle of left arm
            leftAngle = calcAngle(left_wrist, left_elbow, left_shoulder)

            # Visualize angle on image
            cv2.putText(image, str(leftAngle), 
                        tuple(np.multiply(left_elbow, [webcam_width, webcam_height]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        # Rendering detections on frame
        # Passing through image, landmarks from results, which landmark is connected to which, joint drawing spec, connection drawing spec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(252, 80, 3), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(3, 177, 252), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

## Counting Curls

We now have all the pieces to detect and count bicep curls for one arm. We can first find the coordinates of the wrist, elbow, and shoulder. Then, we can calculate the angle at the elbow. We can set a certain threshold for when a curl is in the "down" phase and when it is in the "up" phase. The transition from "down" to "up" can be reflected in incrementing the counter by one.

In [37]:
cap = cv2.VideoCapture(0)

# Variables for counting curls
left_counter = 0
left_stage = None

# Setting up a MediaPipe instance
with mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change cv BGR image to mp RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Making detections
        results = pose.process(image)

        # Turning RGB back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Grab coordinates of left wrist, left elbow, and left shoulder
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Calculate angle of left arm
            leftAngle = calcAngle(left_wrist, left_elbow, left_shoulder)

            # Visualize angle on image
            cv2.putText(image, str(leftAngle), 
                        tuple(np.multiply(left_elbow, [webcam_width, webcam_height]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Logic to know when to count a curl
            if leftAngle > 160:
                left_stage = 'down'
            if leftAngle < 70 and left_stage == 'down':
                left_counter += 1
                left_stage = 'up'
                print(left_counter)

        except:
            pass

        # Rendering the curl counter on the image feed
        cv2.rectangle(image, (0, 0), (400, 100), (244, 177, 16), -1)  # Creating a box over the image

        # Updating box with information about rep count
        cv2.putText(image, 'Left Arm Reps', (15, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(left_counter), (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Updating box with information about rep stage
        cv2.putText(image, 'Left Stage', (180, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, left_stage, (170, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Rendering detections on frame
        # Passing through image, landmarks from results, which landmark is connected to which, joint drawing spec, connection drawing spec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(252, 80, 3), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(3, 177, 252), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

## Extending Final Code To Both Arms

We can finally extend the code to both arms, using information about the following joints: left wrist, left elbow, left shoulder, right wrist, right elbow, and right shoulder.

In [6]:
webcam_width = 1280
webcam_height = 720

In [7]:
cap = cv2.VideoCapture(0)

# Variables for counting curls
left_counter, right_counter = 0, 0
left_stage, right_stage = None, None

# Setting up a MediaPipe instance
with mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Change cv BGR image to mp RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Making detections
        results = pose.process(image)

        # Turning RGB back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extracting landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Grab coordinates of left wrist, left elbow, and left shoulder
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Grab coordinates of right wrist, right elbow, and right shoulder
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Calculate angle of left arm and right arm
            leftAngle = calcAngle(left_wrist, left_elbow, left_shoulder)
            rightAngle = calcAngle(right_wrist, right_elbow, right_shoulder)

            # Visualize angles on image
            cv2.putText(image, str(leftAngle), 
                        tuple(np.multiply(left_elbow, [webcam_width, webcam_height]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(rightAngle), 
                        tuple(np.multiply(right_elbow, [webcam_width, webcam_height]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Logic to know when to count a curl
            if leftAngle > 160:
                left_stage = 'down'
            if leftAngle < 70 and left_stage == 'down':
                left_counter += 1
                left_stage = 'up'
            if rightAngle > 160:
                right_stage = 'down'
            if rightAngle < 70 and right_stage == 'down':
                right_counter += 1
                right_stage = 'up'

        except:
            pass


        # Rendering the curl counter on the image feed for left arm
        cv2.rectangle(image, (0, 0), (400, 100), (183, 159, 0), -1)  # Creating a box over the image

        # Updating box with information about rep count
        cv2.putText(image, 'Left Arm Reps', (15, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(left_counter), (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Updating box with information about rep stage
        cv2.putText(image, 'Left Stage', (180, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, left_stage, (170, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        

        # Rendering the curl counter on the image feed for right arm
        cv2.rectangle(image, (webcam_width-400, 0), (webcam_width, 100), (183, 159, 0), -1)  # Creating a box over the image

        # Updating box with information about rep count
        cv2.putText(image, 'Right Arm Reps', (15 + webcam_width - 400, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(right_counter), (10 + webcam_width - 400, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Updating box with information about rep stage
        cv2.putText(image, 'Right Stage', (180 + webcam_width - 400, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, right_stage, (170 + webcam_width - 400, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Rendering detections on frame
        # Passing through image, landmarks from results, which landmark is connected to which, joint drawing spec, connection drawing spec
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(73, 74, 254), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(183, 159, 0), thickness=2, circle_radius=2))

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
